In [1]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from skimage.feature import hog
# NOTE: the next import is only valid 
# for scikit-learn version <= 0.17
# if you are using scikit-learn >= 0.18 then use this:
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label

import pickle
import random

from utils import *

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# SVM SVC parameters

cspace='YCrCb'
#cspace='HLS'

spatial = 32
histbin = 32
spatial_size = (spatial,spatial)
hist_bins = histbin

orient=12
pix_per_cell=8
cell_per_block=2

hog_channel = 0

In [ ]:
# Read in cars and notcars
# Read in our vehicles and non-vehicles
dir = '../../../CarND-Vehicle-Detection/non-vehicles/**/*.png'

#images = glob.glob('*.jpeg')
images = glob.glob(dir)
cars = []
notcars = []


for image in images:
    if 'image' in image or 'extra' in image:
        notcars.append(image)
#    else:
#        cars.append(image)

print(len(notcars))

cardir = '../../../CarND-Vehicle-Detection/vehicles/**/*.png'

#images = glob.glob('*.jpeg')
carimages = glob.glob(cardir)

for image in carimages:
    cars.append(image)
    
print(len(cars))

8968
8792


In [ ]:
car_features = extract_features(cars, cspace=cspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 256), hog_channel = hog_channel,orient=orient,
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block)
notcar_features = extract_features(notcars, cspace=cspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 256),hog_channel = hog_channel,
                                   orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block)

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using spatial binning of:',spatial,
    'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

In [ ]:
pyramid = [((64, 64),  [400, 500]),
           ((96, 96),  [400, 500]),
           ((128, 128),[450, 578]),
           ((192, 192),[450, None]),
#             ((256, 256),[450, None])
      ]
image_size = (720, 1280)
windows = create_windows(pyramid, image_size)

In [ ]:
images_box = []
all_dist = []

for image in range(6):
    
    prefix = 'test_images/test'
    
    test_img = mpimg.imread( prefix + str(image+1) + '.jpg')


    test, box_list, pred_dist = find_cars(test_img, windows, svc, image_size= (64,64), cutoff=0, 
                            cspace=cspace, spatial_size=spatial_size,  hist_bins=hist_bins, 
                            hog_channel=hog_channel,  orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block,car_image=True)
    images_box.append(test)
    all_dist.append(pred_dist)

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(20,18))
ax1.imshow(images_box[0])
ax1.set_title('0', fontsize=10)
#ax2.imshow(image)
ax2.imshow(images_box[1])
ax2.set_title('1', fontsize=10)

ax3.imshow(images_box[2])
ax3.set_title('2', fontsize=10)

ax4.imshow(images_box[3])
ax4.set_title('3', fontsize=10)

ax5.imshow(images_box[4])
ax5.set_title('4', fontsize=10)

ax6.imshow(images_box[5])
ax6.set_title('5', fontsize=10)

In [ ]:
#print(np.array(all_dist))
#print((np.array(all_dist)).reshape(1,-1))

reshaped =(np.array(all_dist)).reshape(1,-1)
plt.plot(reshaped.T)